<a href="https://colab.research.google.com/github/Ivan137950/Russian-Language-Toxic-Comments/blob/main/Gas_Myas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Russian Language Toxic Comments
https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments/data

In [ ]:
!install-update pip
!pip install bertopic
# !pip install optuna

In [70]:
# import optuna

---
# Скачаем данные в гугл коллаб

---

In [72]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d blackmoon/russian-language-toxic-comments
!unzip russian-language-toxic-comments.zip -d /content/datafr

In [ ]:
df = pd.DataFrame(pd.read_csv("/content/datafr/labeled.csv")) # Итоговая бд
df.head()

---

# ***Предварительный анализ данных.***
Посмотрим на структуру имеющихся данных, чтобы понять, как с ними лучше работать.

---

In [ ]:
ax = sns.histplot(df['toxic'], bins=2, color='purple') #I love purple

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom')

ax.set_xticks([1/4, 3/4])
ax.set_xticklabels(['Нейтральные', 'Негативные'])

plt.ylabel('Количество')
plt.xlabel('Комментарии')
plt.title('Гистограмма')
plt.show()

Видно, что негативных комментариев значительно меньше, и ,при неправильном разеделении датасета на тестовые и проверочные данные, классификатор будет выдавать только 0. Теперь займёся самими комментариями.

In [ ]:
import re
df['clean-text'] = df['comment'].str.lower()
df['clean-text'] = [re.sub(r'[,.!?/\"-=+%*^$<>#():;]', " " , text[:-1]) for text in df['clean-text']]
df['clean-text'] = df['clean-text'].str.split()
df.head()

Рассмотрим топ 25 самых популярных слов




In [ ]:
from collections import Counter

count_dict = Counter()
for i in range(len(df)):
    count_dict.update(df['clean-text'][i])

count_dict = dict(sorted(count_dict.items(),
                         key=lambda item: item[1],
                         reverse=True))

total_amount = sum(count_dict.values())
count_dict_avg = {item[0]:
               item[1] / total_amount
               for item in count_dict.items()}
# count_dict

In [ ]:
keys = list(count_dict.keys())[:100]
values = list(count_dict.values())[:100]

# Создание гистограммы
fig, ax = plt.subplots(figsize=(12, 6))
plt.bar(keys[:25], values[:25])
plt.xlabel("слова")
plt.ylabel("Частота")
plt.title("Частота слов в сообщениях")
plt.grid(True)

---
Видно, что на первых позициях стоят слова, которые, очевидно, одинаково часто встречаются как в грубых, так и в самых невинных комментариях. Давайте найдём те слова, которые одинаково часто встречаются и там, и там, и те слова, которые наоборот, встрчеаются в одном из случаев сильно чаще, чем в другом. Так как негативных коментариев вдвое меньше, буду брать не абсолютные значения, а относительные.

---

In [ ]:
#Разбиваю df на 2 части
comments_neg = df[df['toxic'] == 1]['clean-text']
comments_pos = df[df['toxic'] == 0]['clean-text']
comments_pos.shape, comments_neg.shape

In [ ]:
#Сколько раз каждое слово попало в каждую группу
count_dict_pos = Counter()
for com in comments_pos:
    count_dict_pos.update(com)
count_dict_pos = dict(sorted(count_dict_pos.items(),
                             key=lambda item: item[1],
                             reverse=True))

count_dict_neg = Counter()
for com in comments_neg:
    count_dict_neg.update(com)

count_dict_neg = dict(sorted(count_dict_neg.items(),
                             key=lambda item: item[1],
                             reverse=True))
# count_dict_neg

In [ ]:
#Сколько раз каждое слово попало в каждую группу в процентном соотношении
pos_total_amount = sum(count_dict_pos.values())
neg_total_amount = sum(count_dict_neg.values())

avg_dic_pos = {item[0]:
               item[1] / pos_total_amount
               for item in count_dict_pos.items()}

avg_dic_neg = {item[0]:
               item[1] / neg_total_amount
               for item in count_dict_neg.items()}

In [ ]:
'''
Найдём слова, вероятность которых найти в негативных комментариях сильно
меньше, сильно больше и приблизительно равна вероятности найти их в позитивных,
а также выделим слова, которые встречаются только в одном из классов
'''

pos_set = set(item[0] for item in avg_dic_pos.items())
neg_set = set(item[0] for item in avg_dic_neg.items())
pos_only_arr = list(pos_set - neg_set) #есть только в позитивных комментариях
neg_only_arr = list(neg_set - pos_set) #есть только в негативных комментариях
neg_pos_arr = list(neg_set & pos_set)  #есть в любых комментариях
len(pos_only_arr), len(neg_only_arr), len(neg_pos_arr)

In [ ]:
'''
И теперь для каждого множества из предыдущего пункта зададим таблицы
'''
#встречаются только в позитивных комментариях
tot_avg = [count_dict_avg[pos] for pos in pos_only_arr]
pos_avg = [avg_dic_pos[pos] for pos in pos_only_arr]

pos_df = df[df['toxic'] == 0]
pos_words = [word for arr in pos_df['clean-text'] for word in arr]
word_counts = Counter(pos_words)
pos_amount = [word_counts.get(word, 0) for word in pos_only_arr]

pos_only_df = pd.DataFrame({"words": pos_only_arr,
                            "percantage-in-all-words": tot_avg,
                            "percantage-in-pos-words": pos_avg,
                            "amount-in-messages": pos_amount})
pos_only_df = pos_only_df.sort_values(by="amount-in-messages", ascending=False)
pos_only_df

In [ ]:
#встречаются только в негативных комментариях
tot_avg = [count_dict_avg[neg] for neg in neg_only_arr]
neg_avg = [avg_dic_neg[neg] for neg in neg_only_arr]

neg_df = df[df['toxic'] == 1]
neg_words = [word for arr in neg_df['clean-text'] for word in arr]
word_counts = Counter(neg_words)
neg_amount = [word_counts.get(word, 0) for word in neg_only_arr]

neg_only_df = pd.DataFrame({"words": neg_only_arr,
                            "percantage-in-all-words": tot_avg,
                            "percantage-in-neg-words": neg_avg,
                            "amount-in-messages": neg_amount})
neg_only_df = neg_only_df.sort_values(by="amount-in-messages", ascending=False)
neg_only_df

In [ ]:
#встречаются и в позитивных, и в негативных комментариях
neg_pos_pos = [avg_dic_pos[pos] for pos in neg_pos_arr]
neg_pos_neg = [avg_dic_neg[neg] for neg in neg_pos_arr]
neg_to_pos  = [neg_pos_pos[i] / neg_pos_neg[i] for i in range(len(neg_pos_arr))]
pos_to_neg  = [1 / ntp for ntp in neg_to_pos]
total_dict = [count_dict_avg[elem] for elem in neg_pos_arr]


#количество негативных комментариев со словами из npa
neg_df = df[df['toxic'] == 1]
neg_words = [word for arr in neg_df['clean-text'] for word in arr]
word_counts = Counter(neg_words)
neg_amount_npa = [word_counts.get(word, 0) for word in neg_pos_arr]


#количество позитивных комментариев со словами из npa
pos_df = df[df['toxic'] == 0]
pos_words = [word for arr in pos_df['clean-text'] for word in arr]
word_counts = Counter(pos_words)
pos_amount_npa = [word_counts.get(word, 0) for word in neg_pos_arr]



neg_pos_df = pd.DataFrame({"words": neg_pos_arr,
                           "positive": neg_pos_pos,
                           "negative": neg_pos_neg,
                           "total": total_dict,
                           "more-positive": neg_to_pos,
                           "more-negative": pos_to_neg,
                           "pos-comments-amount": pos_amount_npa,
                           "neg-comments-amount": neg_amount_npa
                           })


neg_pos_df["all-comments-amount"] = neg_pos_df['neg-comments-amount'] + neg_pos_df['pos-comments-amount']
neg_pos_df =  neg_pos_df.sort_values(by="all-comments-amount", ascending=False)
neg_pos_df

Посмотрим на гистограммы, в которых колонки отвечают за количество слов в том или ином наборе

In [ ]:
# Функция распределения для amount-in-messages в pos_only_df
pos_hist_df = pos_only_df['amount-in-messages'].apply(
                                              lambda x: x if x < 11 else 11)

pos_hist_arr = np.sort(np.array([phd for phd in pos_hist_df]))
pos_hist_arr = np.array([
                    str(phd) if phd < 11 else ">10"  for phd in pos_hist_df])

fig, ax = plt.subplots(figsize=(18, 8))
hist_plot = sns.histplot(pos_hist_arr,
                        binwidth=1,
                        stat='density',
                        cumulative=True,
                        ax=ax)
plt.xlim(-1, 11)


plt.xlabel("частота")
plt.ylabel("amount in messages")
plt.title("распределение amount-in-messages для позитивных комментариев")
for p in hist_plot.patches:
    height = p.get_height()
    if height > 0:
        ax.text(
            p.get_x() + p.get_width() / 2,
            height,
            f'{height:.2f}',
            ha='center',
            va='bottom'
        )

plt.show()

In [ ]:
# Функция распределения для amount-in-messages в neg_only_df
neg_hist_df = neg_only_df['amount-in-messages'].apply(
                                              lambda x: x if x < 11 else 11)

neg_hist_arr = np.sort(np.array([phd for phd in neg_hist_df]))
neg_hist_arr = np.array([
                    str(phd) if phd < 11 else ">10"  for phd in neg_hist_df])

fig, ax = plt.subplots(figsize=(18, 8))
hist_plot = sns.histplot(neg_hist_arr,
                        binwidth=1,
                        stat='density',
                        cumulative=True,
                        ax=ax)
plt.xlim(-1, 11)


plt.xlabel("частота")
plt.ylabel("amount in messages")
plt.title("распределение amount-in-messages для негативных комментариев")
for p in hist_plot.patches:
    height = p.get_height()
    if height > 0:
        ax.text(
            p.get_x() + p.get_width() / 2,
            height,
            f'{height:.2f}',
            ha='center',
            va='bottom'
        )

plt.show()

Как можно видеть из этих графиков, львиная доля "слов-маркеров", по которым можно было бы отличать негативные комментарии от позитивных существуют в количестве менее 10 экземпляров, большая часть вообще существует в единственном экземпляре. Более того, даже самые популярные слова стречаются менее чем в 70 сообщениях (при общем объеме датасета в 10 000+ сообщений). Т.о. классифицировать данные просто по "маркерам" относительно бессмысленное занятие, так как при расширении датасета могут появиться сообщения без "слов-маркеров".

Заметим также, что в neg_pos_df наиболее часто встречающиеся в комментариях слова встречаются практически с одинаковой вероятностью как в положительных, так и в отрицательных комментариях. Соответственно самые частовстречающиеся и самые одинаково распределённые слова, такие как "на", "и",... можно удалить как шум.

In [ ]:
stop_words_df = neg_pos_df[(neg_pos_df['all-comments-amount'] > 1500)&
                           (neg_pos_df['more-negative'] > 0.75)&
                           (neg_pos_df['more-positive'] > 0.75)]
stop_words = set(stop_words_df["words"])
#Слова, которые можно будет не учитывать
# stop_words

---
# ***Выполним тематическое моделирование***.

 ***BERTopic***. Выбор обусловлен тем, что весь набор данных достаточно большой для этой модели, однако сами тексты довольно короткие, из-за чего ряд других моделей, например, ***LDA*** покажет худшие результаты, в сравнении с ***BERTopic***.

---


In [ ]:
def clean_from_stop_words(text):
    approved = set(text) - stop_words
    text = " ".join([t for t in text if t in approved])
    return text


berk_text = df['clean-text'].apply(clean_from_stop_words)
df['for-berk'] = berk_text

In [ ]:
!pip install huggingface_hub[hf_xet] #Вроде как так должно работать быстрее

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
# import matplotlib.pyplot as plt

embedding_model = SentenceTransformer("ai-forever/sbert_large_mt_nlu_ru")
topic_model = BERTopic(
    language="russian",
    umap_model=umap.UMAP(n_neighbors=2, min_dist=0.2, n_components=2, metric='cosine'),
    hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=5, metric='chebyshev')
)

topics, probs = topic_model.fit_transform(berk_text.to_list())


In [ ]:
# Посмотрим таблицу с темами
topic_info = pd.DataFrame(topic_model.get_topic_info())
topic_info

In [ ]:
topic_model.visualize_topics()

# ***Варианты  дальнейшего улучшения тематического моделирования***


1.   Подбор параметров в **umap_model** и **hdbscan_model** для уменьшения количества "мусорных" данных.
2.  Можно попробовать *Мультикорпусное сравнение*: обучить на разных выборках и сравнить наборы.



---

# ***Обучение классификатора.***

Построим модель в *PyTorch*.



*   В качестве модели нейронной сети будет использоваться *GRU*, (Gated Recurrent Unit), так как он является одним из типов рекуррентных нейронных сетей (*RNN*), предназначенных для обработки последовательных данных, таких как текст. *GRU* является более легковесной и вычислительно эффективной моделью по сравнению с более сложным *LSTM*, однако в задачах классификации текса чатсо показывает результаты на уровне *LSTM*.

*  В качестве критерия оценки качества будет использоваться *accuracy*, так как эта метрика используется в большинстве задач классификации и при этом легко интерпретируется: доля правильных ответов. Поэтому выбор пал  именно на неё, хотя *Precision*, *Recall* или *F1-score* также могли бы использоваться.

*  В качестве функции потерь берёся *BCEWithLogitsLoss*, так как на данный момент считается предпочтительной в задачах бинарной классификации.


---


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.nn import functional as F


In [ ]:
#Для работы с lstm моделями нужно подавать не сообщения, а массивы слов
df['for-lstm'] =  df['for-berk'].str.split()
df

In [ ]:
from sklearn.model_selection import train_test_split
texts =  df['for-lstm'].to_list()
labels =  df['toxic'].to_list()
X_train, X_temp, y_train, y_temp = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp
)

Головная боль -- сообщения все разной длины. Нужно использовать padding. В качестве MaxLen буду брать 95-й порцентиль.

In [ ]:
lens = [len(x) for x in X_train]
max_len = int(np.round(np.percentile(lens, 95)) + 1)
max_len

In [ ]:
#Прелесть keras -- совместимость с numpy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
 def tokenizing_fn(X_train, X_val, X_test, num_words=2000):
    tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    #tokenizing
    train_seq = tokenizer.texts_to_sequences(X_train)
    val_seq = tokenizer.texts_to_sequences(X_val)
    test_seq = tokenizer.texts_to_sequences(X_test)

    return train_seq, val_seq, test_seq

def padding_fn(train_seq, val_seq, test_seq):
    # 3. Padding
    train_pad_seq = pad_sequences(train_seq, padding='post', maxlen=max_len)
    val_pad_seq = pad_sequences(val_seq, padding='post', maxlen=max_len)
    test_pad_seq = pad_sequences(test_seq, padding='post', maxlen=max_len)

    return train_pad_seq, val_pad_seq, test_pad_seq



def tokenizing_and_padding_fn(X_train, X_val, X_test, num_words=2000):
    train_seq, val_seq, test_seq = tokenizing_fn(X_train, X_val, X_test, num_words=num_words)
    return padding_fn(train_seq, val_seq, test_seq)

**GRU** model

In [ ]:
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx, num_layers=2):
        super(GRUClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        # self.dropout = nn.Dropout(0.2)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, dropout=0.1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        # embedded = self.dropout(embedded)
        gru_out, _ = self.gru(embedded)

        hidden = gru_out[:, -1, :]

        output = self.fc(hidden)
        # output = torch.sigmoid(output)
        return output

Datasets & Dataloaders

In [ ]:
def do_dataloaders(train_data, train_labels, val_data,
                   val_labels, test_data, test_labels, device='cpu'):

    train_data = torch.tensor(train_data, dtype=torch.long).to(device)
    val_data = torch.tensor(val_data, dtype=torch.long).to(device)
    test_data = torch.tensor(test_data, dtype=torch.long).to(device)

    train_labels = torch.tensor(train_labels, dtype=torch.float32).to(device)
    val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device)
    test_labels = torch.tensor(test_labels, dtype=torch.float32).to(device)

    train_dataset = TensorDataset(train_data, train_labels)
    val_dataset   = TensorDataset(val_data, val_labels)
    test_dataset  = TensorDataset(test_data, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader


Train and validate model

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import time

In [ ]:


def calculate_accuracy(predictions, labels):
    predicted_labels = (predictions > 0.5).float()
    correct = (predicted_labels == labels.view(-1, 1)).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

def train_and_validate(model, train_loader, val_loader, epochs, device='cpu'):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    acc_train_hist = []
    acc_val_hist = []
    start_time = time.time()
    for epoch in range(epochs):
        if epoch > 2:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 0.0005
        # Тренировка
        model.train()
        train_correct = 0
        train_total = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()

            train_accuracy = calculate_accuracy(torch.sigmoid(predictions), batch_y)
            train_correct += (train_accuracy * batch_y.size(0))
            train_total += batch_y.size(0)

        train_accuracy = train_correct / train_total
        acc_train_hist.append(train_accuracy)

        # Валидация
        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                predictions = model(batch_X)
                val_accuracy = calculate_accuracy(torch.sigmoid(predictions), batch_y)
                val_correct += (val_accuracy * batch_y.size(0))
                val_total += batch_y.size(0)

        val_accuracy = val_correct / val_total
        acc_val_hist.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{epochs}], "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}")
    end_time = time.time()
    print(f"TIME is {end_time-start_time:.4f} on device={device}")
    return acc_train_hist, acc_val_hist, model


In [ ]:
def test_accuracy_score(test_dataloader, model, device):
    model.eval()
    test_correct = 0
    test_total = 0
    for batch_X, batch_y in test_dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        predictions = model(batch_X)
        train_accuracy = calculate_accuracy(torch.sigmoid(predictions), batch_y)
        test_correct += (train_accuracy * batch_y.size(0))
        test_total += batch_y.size(0)

    test_accuracy = test_correct / test_total
    print(f"TEST accuracy = {test_accuracy:.4f} on device={device}")


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
histories = {}

def modeling(X_train, X_val, X_test, device):
    vocab_size = 2500
    # padding
    train_pad_seq, val_pad_seq, test_pad_seq = tokenizing_and_padding_fn(X_train, X_val, X_test, num_words=vocab_size)
    # dataloaders
    train_loader, val_loader, test_loader = do_dataloaders(train_pad_seq, y_train, val_pad_seq, y_val, test_pad_seq, y_test, device=device)
    # model
    model = GRUClassifier(vocab_size=vocab_size, embedding_dim=128, hidden_dim=64, output_dim=1, pad_idx=0).to(device)
    # training
    train_hist, val_hist, model = train_and_validate(model, train_loader, val_loader, epochs=10, device=device)
    # testing
    test_accuracy_score(test_loader, model, device)
    return train_hist, val_hist


train_hist, val_hist = modeling(X_train, X_val, X_test, 'cuda')
histories['cuda train'] = train_hist
histories['cuda val'] =  val_hist
train_hist, val_hist = modeling(X_train, X_val, X_test, 'cpu')
histories['cpu train'] = train_hist
histories['cpu val'] =  val_hist

Визуализация обучения

In [ ]:
plt.plot(histories['cpu train'], label="Train Accuracy")
plt.plot(histories['cpu val'] , label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("График Accuracy для расчётов на CPU")
plt.legend()
plt.show()

In [ ]:
plt.plot(histories['cuda train'], label="Train Accuracy")
plt.plot(histories['cuda val'] , label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("График Accuracy для расчётов на CUDA")
plt.legend()
plt.show()

---

# *Что можно улучшить?*


1.   Использование предобученных эмбеддингов, таких как *Word2Vec* или *BERT*.
2.   Увеличение данных (Data Augmentation).
3.   Вместо GRU можно попробовать более сложные архитектуры, такие как *transformers*.
4.   Auto ML. Использовать методы подбора гипперпараметров, такие как optuna лдя выбора оптимальных параметров *learning rate*, *vocab_size*, *dropout*, *embedding_dim*.
4.   Дополнительная оптимизация *learning rate*: требуется вручную уменьшать его, чтобы избегать переобучения


---